In [ ]:
pip install fsspec==2024.2.0

In [ ]:
pip install unidecode

In [ ]:
%load_ext autoreload
%autoreload 2

# Bibliotecas padrão
import collections
import itertools
import json
import pickle as pkl
import random
import re
import numpy as np
import pandas as pd
import scipy

from transformers import AutoTokenizer, AutoModel

# Visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Outras bibliotecas
import unidecode

# Processamento de linguagem natural
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from spacy.lang.pt import Portuguese
import spacy

# Importando bibliotecas necessárias
import pandas as pd
import pickle as pkl
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
df = pd.read_csv("CommentsReddit.csv")

In [ ]:
#remocao de urls etc com regex

df.Comentário = list(
    map(
        lambda x: re.sub(
            r"[(http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)",
            "",
            x,
        ),
        df.Comentário.values,
    )
)

df.Comentário = list(
    map(
        lambda x: re.sub(
            r"(\n)|(\t)|(\s{2,})",
            " ",
            x,
        ),
        df.Comentário.values,
    )
)

In [ ]:
#adicionando palavras indesejaveis nos topicos

stop_list = ['kkkkk','kkkkkkk','kkkkkk','kkkk','kkkkkkkk','kkkkkkkkk','kkk','kkkkkkkkkk']
stop_words=stopwords.words('portuguese')
stop_words.extend(stop_list)

# VADER/LeiA

In [ ]:
from leia import SentimentIntensityAnalyzer
s = SentimentIntensityAnalyzer()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
sent_i = SentimentIntensityAnalyzer()
def vadar_sentiment(text):
    return sent_i.polarity_scores(text)['compound']

In [ ]:
def categorise_sentiment(sentiment, neg_threshold=-0.05, pos_threshold=0.05):
    if sentiment < neg_threshold:
        label = 'negative'
    elif sentiment > pos_threshold:
        label = 'positive'
    else:
        label = 'neutral'
    return label

In [ ]:
df['coef'] = df['Comentário'].apply(vadar_sentiment)

In [ ]:
df['sentimentoLeia'] = df['coef'].apply(categorise_sentiment)

In [ ]:
# Definir o mapeamento
label_mapping = {"negative": 0, "positive": 2, "neutral": 1}

# Aplicar o mapeamento apenas às colunas especificadas
columns_to_encode = ['sentimentoLeia']
df = df.copy()
df[columns_to_encode] = df[columns_to_encode].applymap(lambda x: label_mapping[x])

In [ ]:
annotator1_g1_labels =  df['class_result'].tolist()
annotator2_g2_labels =  df['sentimentoLeia'].tolist()

In [ ]:
from sklearn.metrics import cohen_kappa_score

# Calculate Cohen's Kappa between annotators 1 and 2
kappa_1_2 = cohen_kappa_score(
    annotator1_g1_labels,
    annotator2_g2_labels
)

kappa_1_2

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(annotator1_g1_labels, annotator2_g2_labels))

# Pysentimiento

In [ ]:
pip install pysentimiento

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns
import re

stop_words = stopwords.words('portuguese')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#\w+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.strip()
    return text

def remove_stopwords(text, stop_words):
    words = text.split()
    words = [word for word in words if not word in stop_words]
    return " ".join(words)

In [ ]:
df['Comentário'] = df['Comentário'].apply(remove_stopwords, stop_words=stop_words)
df['Comentário'] = df['Comentário'].apply(clean_text)

In [ ]:
from pysentimiento import create_analyzer

analyzer = create_analyzer(task="sentiment", lang="pt")

def get_top_sentiment(comment):
    analysis = analyzer.predict(comment)
    return max(analysis.probas, key=analysis.probas.get)

df['pysentimiento'] = df['Comentário'].apply(get_top_sentiment)

In [ ]:
df['pysentimiento'] = df['pysentimiento'].replace({'POS': 2, 'NEG': 0, 'NEU': 1})

In [ ]:
annotator1_g1_labels =  df['class_result'].tolist()
annotator2_g2_labels =  df['pysentimiento'].tolist()

In [ ]:
kappa_1_2 = cohen_kappa_score(
    annotator1_g1_labels,
    annotator2_g2_labels
)

kappa_1_2

In [ ]:
print(classification_report(annotator1_g1_labels, annotator2_g2_labels))

# XLM-RoBERTa

In [ ]:
from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoConfig
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import pipeline
import numpy as np
from scipy.spatial.distance import cosine
from collections import defaultdict
import urllib
import numpy as np
from scipy.special import softmax
from sklearn.metrics import classification_report
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from scipy.special import softmax

# Carregue o modelo e o tokenizador
roberta = "cardiffnlp/xlm-roberta-base-tweet-sentiment-pt"
model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)
model.config.max_position_embeddings

labels = ['Negative', 'Neutral', 'Positive']

def get_sentiment_probabilities(tweet_text):
    try:
        encoded_tweet = tokenizer(tweet_text, return_tensors='pt', truncation=True, padding=True, max_length=512)

        with torch.no_grad():
            outputs = model(**encoded_tweet)
            predictions = outputs.logits
            probabilities = softmax(predictions, axis=1).tolist()[0]

        label_probabilities = {label: prob for label, prob in zip(labels, probabilities)}
        return label_probabilities
    except Exception as e:
        print("Error:", e)
        return None

In [ ]:
df['sentiment_probs'] = df['Comentário'].apply(get_sentiment_probabilities)

In [ ]:
def get_max_sentiment(sentiment_dict):
    max_prob = max(sentiment_dict.values())
    return [label for label, prob in sentiment_dict.items() if prob == max_prob]

In [ ]:
df['RoBERTa'] = df['sentiment_probs'].apply(get_max_sentiment)
# Explodir a coluna 'sentimento' e contar os valores
sentimento_counts = df.explode('RoBERTa')['RoBERTa'].value_counts()

In [ ]:
df = df.explode('RoBERTa')

In [ ]:
# Definir o mapeamento
label_mapping = {"Negative": 0, "Positive": 2, "Neutral": 1}

# Aplicar o mapeamento apenas às colunas especificadas
columns_to_encode = ['RoBERTa']
df = df.copy()
df[columns_to_encode] = df[columns_to_encode].applymap(lambda x: label_mapping[x])

In [ ]:
annotator1_g1_labels =  df['class_result'].tolist()
annotator2_g2_labels =  df['RoBERTa'].tolist()

In [ ]:
kappa_1_2 = cohen_kappa_score(
    annotator1_g1_labels,
    annotator2_g2_labels
)

kappa_1_2

In [ ]:
print(classification_report(annotator1_g1_labels, annotator2_g2_labels))